In [ ]:
!pip3 install python-google-places

In [ ]:
!pip3 install Unidecode

In [1]:
from pymongo import MongoClient
from googleplaces import GooglePlaces, types, lang
from unidecode import unidecode 
import googlemaps
from datetime import datetime
import json
import time
import requests
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
def searchPlaces(query):
    API_key = os.getenv('key')
    endpoint_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    places = []
    params = {
        'query': query,
        'location': '52.37021, 4.895168',
        'radius': 5000,
        'key': API_key
    }
    res = requests.get(endpoint_url, params = params)
    results =  json.loads(res.content)
    places.extend(results['results'])
    time.sleep(2)
    while "next_page_token" in results:
        params['pagetoken'] = results['next_page_token'],
        res = requests.get(endpoint_url, params = params)
        results = json.loads(res.content)
        places.extend(results['results'])
        time.sleep(2)
    return places

In [3]:
starbucks = searchPlaces('Starbucks')

In [4]:
len(starbucks)

34

In [5]:
def getLocation(companies):
    location =[]
    for x in range(len(companies)):
        longitude = companies[x]['geometry']['location']['lng']
        latitude = companies[x]['geometry']['location']['lat']
        name=companies[x]['name']
        loc = {
            'name': name,
            'type':'Point',
            'coordinates':[latitude, longitude]
        }
        location.append(loc)
    return location

In [6]:
starbucks[0]['geometry']

{'location': {'lat': 52.36448240000001, 'lng': 4.8836336},
 'viewport': {'northeast': {'lat': 52.36607492989272,
   'lng': 4.885273579892721},
  'southwest': {'lat': 52.36337527010728, 'lng': 4.882573920107277}}}

In [7]:
sbloc = getLocation(starbucks)

In [8]:
print(len(sbloc))
sbloc[0]

34


{'name': 'STARBUCKS COFFEE Leidsestraat Shop',
 'type': 'Point',
 'coordinates': [52.36448240000001, 4.8836336]}

In [9]:
schools = searchPlaces('Primary school')
len(schools)
scloc = getLocation(schools)
print(len(scloc))
scloc[1]

60


{'name': '15ᵗʰ Montessori Public Elementary School Maas and Waal',
 'type': 'Point',
 'coordinates': [52.3433973, 4.899729]}

In [10]:
kindergarten = searchPlaces('kindergarten')
len(kindergarten)
kdloc = getLocation(kindergarten)
print(len(kdloc))
kdloc[2]

60


{'name': 'Kindergarden Kinderdagverblijf Amsterdam Villa Vondel',
 'type': 'Point',
 'coordinates': [52.35313420000001, 4.8576681]}

In [11]:
night_club = searchPlaces('night club')
len(night_club)
ncloc = getLocation(night_club)
print(len(ncloc))
ncloc[3]

60


{'name': 'Blue Ivy Nightclub',
 'type': 'Point',
 'coordinates': [52.3660594, 4.8996878]}

In [12]:
vegan_restaurant = searchPlaces('vegan restaurant')
len(vegan_restaurant)
vrloc = getLocation(vegan_restaurant)
print(len(vrloc))
vrloc[4]

60


{'name': 'Loving Hut - Vegan House',
 'type': 'Point',
 'coordinates': [52.3810426, 4.8544336]}

In [13]:
airport = searchPlaces('airport amsterdam')
len(airport)
aloc = getLocation(airport)
print(len(aloc))
aloc[0]

1


{'name': 'Schiphol',
 'type': 'Point',
 'coordinates': [52.3076865, 4.767424099999999]}

In [14]:
gmaps = googlemaps.Client(key=os.getenv("key"))
places_result = gmaps.places('airport, Amsterdam')


In [15]:
places_result['results'][0]['geometry']['location']

{'lat': 52.3076865, 'lng': 4.767424099999999}

In [16]:
%store sbloc #starbucks
%store scloc #schools
%store kdloc #kindergarten
%store ncloc #night club
%store vrloc #vegan restaurant
%store aloc #airport

Stored 'sbloc' (list)
Stored 'scloc' (list)
Stored 'kdloc' (list)
Stored 'ncloc' (list)
Stored 'vrloc' (list)
Stored 'aloc' (list)


In [17]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [18]:
db, coll = connectCollection('companies', 'companies')

In [19]:
diccsearch = {'starbucks': sbloc,
              'schools': scloc,
              'kindergarten': kdloc,
              'clubs': ncloc,
              'veganrest': vrloc,
              'airport': aloc}

In [20]:
def insertColl(dictionary, database):
    for k, v in dictionary.items():
        col = database[k]
        col.insert_many(v)
        print("collects inserted")

In [21]:
insertColl(diccsearch, db)

collects inserted
collects inserted
collects inserted
collects inserted
collects inserted
collects inserted


In [22]:
aloc[0]['coordinates'][0]


52.3076865

In [25]:
def getLocation(companies):
    location =[]
    longitude = companies['coordinates'][1]
    latitude = companies['coordinates'][0]
    loc = {
        'type':'Point',
        'coordinates':[latitude, longitude]
    }
    location.append(loc)
    print(location)
    return location

In [26]:
for k, v in diccsearch.items():
    for v1 in v:
        db, coll = connectCollection('companies', k)
        value = {"$set": {'location':getLocation(v1)}}
        query = {"_id": v1['_id']}
        coll.update_many(query, value)

[{'type': 'Point', 'coordinates': [52.36448240000001, 4.8836336]}]
[{'type': 'Point', 'coordinates': [52.37429770000001, 4.8942746]}]
[{'type': 'Point', 'coordinates': [52.36699789999999, 4.8896652]}]
[{'type': 'Point', 'coordinates': [52.3659541, 4.8977405]}]
[{'type': 'Point', 'coordinates': [52.37904899999999, 4.900354099999999]}]
[{'type': 'Point', 'coordinates': [52.37687829999999, 4.9043465]}]
[{'type': 'Point', 'coordinates': [52.3553881, 4.8905951]}]
[{'type': 'Point', 'coordinates': [52.3892232, 4.8375534]}]
[{'type': 'Point', 'coordinates': [52.3493467, 4.8771144]}]
[{'type': 'Point', 'coordinates': [52.33961009999999, 4.8731287]}]
[{'type': 'Point', 'coordinates': [52.36311569999999, 4.8758273]}]
[{'type': 'Point', 'coordinates': [52.3464775, 4.917986099999999]}]
[{'type': 'Point', 'coordinates': [52.3703102, 4.892257499999999]}]
[{'type': 'Point', 'coordinates': [52.32816709999999, 4.8896519]}]
[{'type': 'Point', 'coordinates': [52.3798957, 4.9005785]}]
[{'type': 'Point', '

[{'type': 'Point', 'coordinates': [52.3666451, 4.8923966]}]
[{'type': 'Point', 'coordinates': [52.363632, 4.856074]}]
[{'type': 'Point', 'coordinates': [52.3630419, 4.8849222]}]
[{'type': 'Point', 'coordinates': [52.364062, 4.8846631]}]
[{'type': 'Point', 'coordinates': [52.364748, 4.882211]}]
[{'type': 'Point', 'coordinates': [52.3746525, 4.8970018]}]
[{'type': 'Point', 'coordinates': [52.3649221, 4.8822071]}]
[{'type': 'Point', 'coordinates': [52.3501183, 4.867600599999999]}]
[{'type': 'Point', 'coordinates': [52.3644007, 4.8848357]}]
[{'type': 'Point', 'coordinates': [52.3603552, 4.8806563]}]
[{'type': 'Point', 'coordinates': [52.378287, 4.894546]}]
[{'type': 'Point', 'coordinates': [52.3648119, 4.8818315]}]
[{'type': 'Point', 'coordinates': [52.3718604, 4.895861000000001]}]
[{'type': 'Point', 'coordinates': [52.38713019999999, 4.8700145]}]
[{'type': 'Point', 'coordinates': [52.3648955, 4.882159199999999]}]
[{'type': 'Point', 'coordinates': [52.3856761, 4.8519958]}]
[{'type': 'Point